<a href="https://colab.research.google.com/github/lmencisoe/CDA/blob/main/Taller3/Modelo_Taller3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade pandas-profiling

In [ ]:
!pip install shap

In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')
import xgboost as xgb
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.decomposition import PCA
from pandas_profiling import ProfileReport
import tensorflow as tf
from sklearn.feature_extraction import DictVectorizer
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from keras.models import Sequential
from xgboost import plot_importance
from keras.layers import Dropout, Flatten, Dense, Input, Rescaling, Lambda, MaxPooling2D, Conv2D, AveragePooling2D
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import ConfusionMatrixDisplay, precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection  import GridSearchCV
from matplotlib import pyplot
import seaborn as sns
import shap
import warnings
import re
warnings.filterwarnings('ignore')

In [ ]:
df=pd.read_csv("/content/drive/MyDrive/Taller3_CDA/Data_Taller_3_modelo.csv",sep=";")

In [ ]:
df2 = df.copy()
df2 = df2[df2['NewEmployee']==0]

In [ ]:
df.info()

In [ ]:
X_total = df2.drop(['JobSatisfaction'], axis=1)
Y_total = df2['JobSatisfaction']

In [ ]:
#multi:softprob

#*Entrena modelo base para selección de mejores variables (XgBoost)*

In [ ]:
xgb_base = xgb.XGBClassifier(objective='multi:softprob',
         seed = 2022)

xgb_base.fit(X_total,
         Y_total,  
         eval_metric="merror")

In [ ]:
plot_importance(xgb_base, height = 1, max_num_features = 15)
pyplot.show()

# *Se decide dejar las variables: MonhtlyIncome, MonthlyRate, DailyRate, HourlyRate, Age, DistanceFromHome, PercentSalaryHike y YearsSinceLastPromotion*\\
*Con las demás se decide construir kmeans y grupos para reducir dimensionalidad*

In [ ]:
df3 = df.copy()
df3.shape

In [ ]:
X_total = df3.drop(['JobSatisfaction'], axis=1)

In [ ]:
X_base = X_total[['MonthlyIncome','MonthlyRate', 'DailyRate', 'HourlyRate', 'Age', 'DistanceFromHome', 'PercentSalaryHike', 'YearsSinceLastPromotion','NewEmployee']]
X_resto = X_total.drop(['MonthlyIncome','MonthlyRate', 'DailyRate', 'HourlyRate', 'Age', 'DistanceFromHome', 'PercentSalaryHike', 'YearsSinceLastPromotion'], axis=1)

# **Se construye un PCA con las demás variables (Esto hace parte del bono)**

In [ ]:
pca = PCA(n_components=10)
principalComponents = pca.fit_transform(X_resto)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['pc1', 'pc2','pc3', 'pc4','pc5', 'pc6','pc7', 'pc8','pc9', 'pc10'])

# Se usa un kmeans y método del codo para seleccionar 5 grupos, con el fin de resumir las variables restantes en estos 5 grupos y que sean features del nuevo modelo.

In [ ]:
kmeans_kwargs = {
    "init": "random",
    "n_init": 10,
    "max_iter": 300,
    "random_state": 42,
}

# A list holds the SSE values for each k
sse = []
for k in range(1, 11):
    kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
    kmeans.fit(principalDf)
    sse.append(kmeans.inertia_)

In [ ]:
plt.style.use("fivethirtyeight")
plt.plot(range(1, 11), sse)
plt.xticks(range(1, 11))
plt.xlabel("Number of Clusters")
plt.ylabel("SSE")
plt.show()

In [ ]:
kmeans = KMeans(
     init="random",
     n_clusters=5,
     n_init=10,
     max_iter=300,
     random_state=42)
kmeans.fit(principalDf)
kmeans.labels_[:5]

In [ ]:
X_base['new'] = kmeans.labels_

In [ ]:
dummies_1 = pd.get_dummies(X_base['new'], prefix='cluster_g_').reset_index(drop=True)
X_base = pd.concat([X_base.reset_index(drop=True), dummies_1], axis=1)

In [ ]:
X_base = X_base.drop(columns=['new'])

Nuevas X con las variables seleccionadas y los grupos del kmeans con la reducción de dimensionalidad:

In [ ]:
X_base

In [ ]:
X_base_model = X_base[X_base['NewEmployee'] == 0]
X_base_model = X_base_model.drop(columns=['NewEmployee'])
Y_model = df2[df2['NewEmployee']==0]['JobSatisfaction']
Y_model

In [ ]:
X_base_model

In [ ]:
Y_model = Y_model.apply(int)
Y_model

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_base_model, Y_model, test_size=0.33, random_state=2022)

# **Tunning de hiperparámetros del XGBoost (Esto hace parte del bono)**

In [ ]:
xgb_model = xgb.XGBClassifier()

parameters = {'nthread':[3], #when use hyperthread, xgboost may become slower
              'objective':['multi:softprob'],
              'learning_rate': [0.01,0.05,0.1], #so called `eta` value
              'max_depth': [2,4,6],
              'subsample': [0.1,0.15],
              'colsample_bytree': [0.1,0.15],
              'n_estimators': [40,80], #number of trees, change it to 1000 for better results
              'seed': [2022]}


xgb_grid = GridSearchCV(xgb_model,
                        parameters,
                        cv = 3,
                        n_jobs = 3,
                        verbose=True)

In [ ]:
with tf.device('/device:GPU:0'): 
    xgb_grid.fit(X_train, 
                 y_train,
                 eval_metric="merror",
                 early_stopping_rounds=20,
                 eval_set = [(X_train, y_train),(X_test, y_test)])    

In [ ]:
print(xgb_grid.best_score_)
print(xgb_grid.best_params_)

Mejor modelo del XGBoost:

In [ ]:
xgb_selecc = xgb.XGBClassifier(nthread=3, #when use hyperthread, xgboost may become slower
         objective='multi:softprob',
         learning_rate= 0.05, #so called `eta` value
         max_depth= 4,
         n_estimators = 40,
         subsample= 0.1,
         colsample_bytree= 0.1,
         seed = 2022)

xgb_selecc.fit(X_train,
         y_train,  
         eval_metric="merror",
         early_stopping_rounds=20,
         eval_set = [(X_train, y_train),(X_test, y_test)])

In [ ]:
pred_modelo_train = xgb_selecc.predict(X_train)
pred_modelo_test = xgb_selecc.predict(X_test)

In [ ]:
class_names = ['1','2','3','4']

In [ ]:
print(classification_report(y_train,pred_modelo_train,target_names=list(class_names)))

In [ ]:
print(classification_report(y_test,pred_modelo_test,target_names=list(class_names)))

In [ ]:
accuracy_score(y_train, pred_modelo_train), accuracy_score(y_test, pred_modelo_test)

# **Importancia de variables (este es el modelo seleccionado, más adelante se justifica)**

In [ ]:
plot_importance(xgb_selecc, height = 1, max_num_features = 15)
pyplot.show()

#*Random Forest*

In [ ]:
parameters = {
    'n_estimators': [10,20,30],
    'max_depth': [2,5,7],
    'max_features': ['auto', 'sqrt', 'log2'],
    'min_samples_split': [3,5,7]
}
regr = RandomForestClassifier(random_state=0)

clf = GridSearchCV(regr, 
                   parameters,
                   cv = 5)
clf.fit(X_train, y_train)

In [ ]:
print(clf.best_score_)
print(clf.best_params_)

Mejor RandomForest

In [ ]:
rf_selecc = RandomForestClassifier(
         n_estimators= 30, #so called `eta` value
         max_features = 'auto',
         max_depth=7,
         min_samples_split = 3)

rf_selecc.fit(X_train,
         y_train)

In [ ]:
pred_modelo_train = rf_selecc.predict(X_train)
pred_modelo_test = rf_selecc.predict(X_test)

In [ ]:
print(classification_report(y_train,pred_modelo_train,target_names=list(class_names)))

In [ ]:
print(classification_report(y_test,pred_modelo_test,target_names=list(class_names)))

In [ ]:
accuracy_score(y_train, pred_modelo_train), accuracy_score(y_test, pred_modelo_test)

#*Arbol de decisión*

In [ ]:
parameters = {
    'max_depth': [2,5,7],
    'min_samples_split': [3,5,7],
    'max_features': ['auto', 'sqrt', 'log2']
}
regr_arb = DecisionTreeClassifier(random_state=0)

arbol = GridSearchCV(regr_arb, 
                   parameters,
                   cv = 5)
arbol.fit(X_train, y_train)

In [ ]:
print(arbol.best_score_)
print(arbol.best_params_)

Mejor árbol de decisión:

In [ ]:
arbol_selecc = DecisionTreeClassifier(
         max_depth= 2, #so called `eta` value
         min_samples_split = 3,
         max_features = 'auto')

arbol_selecc.fit(X_train,
         y_train)

In [ ]:
pred_modelo_train = arbol_selecc.predict(X_train)
pred_modelo_test = arbol_selecc.predict(X_test)

In [ ]:
print(classification_report(y_train,pred_modelo_train,target_names=list(class_names)))

In [ ]:
print(classification_report(y_test,pred_modelo_test,target_names=list(class_names)))

In [ ]:
accuracy_score(y_train, pred_modelo_train), accuracy_score(y_test, pred_modelo_test)

#*Red Neuronal*

In [ ]:
y_dict = [{'':str(x)} for x in y_train.tolist()]
y_dict_test = [{'':str(x)} for x in y_test.tolist()]


In [ ]:
vec = DictVectorizer(sparse=False, dtype=int)
M_train = vec.fit_transform(y_dict)
M_test = vec.fit_transform(y_dict_test)

In [ ]:
model = Sequential(name="RedNeuronal")
model.add(Input(shape=(13),name='Capa_Entrada'))
model.add(Dense(16,activation='relu',name='Capa_Densa_3'))
model.add(Dense(8,activation='relu',name='Capa_Densa_4'))
model.add(Dense(len(class_names),activation='softmax',name='Capa_Respuesta'))
model.compile(loss='categorical_crossentropy', optimizer='sgd',metrics=[tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
model.summary()

In [ ]:
my_callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=4)
]

In [ ]:
history = model.fit(X_train, 
        M_train,           
        validation_data=([X_test, M_test]),
        epochs=50,
        callbacks=my_callbacks
    )

In [ ]:
pred_train = model.predict(X_train)
y_pred = list()
for i in range(len(pred_train)):
    y_pred.append((pred_train[i] == pred_train[i].max()).astype(int))

In [ ]:
y_pred = np.array(y_pred)
print(classification_report(M_train,y_pred,target_names=list(class_names)))

In [ ]:
accuracy_score(M_train, y_pred)

In [ ]:
pred_test = model.predict(X_test)
y_pred_test = list()
for i in range(len(pred_test)):
    y_pred_test.append((pred_test[i] == pred_test[i].max()).astype(int))

In [ ]:
y_pred_test = np.array(y_pred_test)
print(classification_report(M_test,y_pred_test,target_names=list(class_names)))

In [ ]:
accuracy_score(M_test, y_pred_test)

# *Se selecciona el XGBoost con los hiperparámetros mostrados anteriormente, ya que no presenta sobreajuste y además, predice de una forma estable las 4 categorías* (Esto se aprecia en el classification_report())

# **Interpretación**
*Se puede apreciar en la matriz de importancia que en el top aparecen tanto la edad como el MonthlyIncome, por lo tanto, son features significativas para el modelo*

In [ ]:
explainer = shap.Explainer(xgb_grid.best_estimator_.predict, X_train)
shap_values = explainer(X_train)

In [ ]:
shap.summary_plot(shap_values, plot_type = 'violin')

Con el shap se puede apreciar claramente que: \\
1. Valores altos de MonthlyRate afectan negativamente el output, mientras que valores bajos lo afectan positivamente. \\
2. Se apreciar también que con la reducción de dimensionalidad, trabajadores en el cluster 0 tienden a estar menos satisfechos que trabajadores que no pertenecen a este cluster. \\
3. Además, es interesante también, ver como a mayor cantidad de años desde la última promoción (YearsSinceLastPromotion) la satisfacción es más baja.

# **Evaluación**

Los siguientes son los acc de los diferentes modelos:
1. XGBoost -> train: 34.7% , test: 33.4%
2. RandomForest -> train: 68.8%, test_28.7%
3. DecisionTree -> train: 32.6%, test: 34.1%
4. Red Neuronal -> train: 33.4%, test: 29.4%

Y se selecciona el XGBoost como se comenta antes, ya que es el único que predice sobre las 4 variables, además de no presentar sobreajuste.

Estrategias:
1. Tomar algún tipo de iniciativa con los trabajadores de mayor edad, ya que esta variable, al incrementar afecta negativamente la satisfacción.
2. Mayor promoción interna, antes de contratación de nuevos empleados.
3. Permitir, de ser posible el trabajo remoto, ya que se aprecia que a mayor distancia del hogar, esto afecta negativamente.

# **Estimación de nuevos empleados**

In [ ]:
X_new_model = X_base[X_base['NewEmployee'] == 1]

In [ ]:
X_new_model = X_new_model.drop(columns=['NewEmployee'])

In [ ]:
pred_modelo_new = xgb_selecc.predict(X_new_model)

In [ ]:
pred_modelo_new

# **Bono: Ya se realizó anteriormente**